<a href="https://colab.research.google.com/github/menhguin/self_explaining_features/blob/main/Minh's_Copy_of_Gemma_SAE_self_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Doc: https://docs.google.com/document/d/16TBV6MCH_zf3MfD8hdK7k51Vkx2OOV-mmhe4YoKfKKE/edit?tab=t.0

Dmitrii Kharlapenko's Colab: https://colab.research.google.com/drive/1_geSRb0oPFTsSoiEhUJoYjCQv3JmK4R_?usp=sharing

Dmitrii Kharlapenko's LessWrong Post: https://www.lesswrong.com/posts/8ev6coxChSWcxCDy8/self-explaining-sae-features

Original SelfIE paper: https://arxiv.org/abs/2403.10949

# Setup

This notebook is sorta-patched as of 16/12/2024.

In [1]:
#@title Install Dependencies
!pip install -q nnsight accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 902.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
#@title HF login
import huggingface_hub
from google.colab import userdata
huggingface_hub.login(userdata.get('HF_READ_TOKEN'))

In [3]:
#@title Load Gemma
from nnsight import LanguageModel
import numpy as np
import torch
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False);

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


In [4]:
#@title Load SAE
use_instruct_sae = False  #@param {type: "boolean"}
if use_instruct_sae:
    !wget -c 'https://huggingface.co/jbloom/Gemma-2b-IT-Residual-Stream-SAEs/resolve/main/gemma_2b_it_blocks.12.hook_resid_post_16384/sae_weights.safetensors?download=true' -O 'sae-it.safetensors'
else:
    !wget -c 'https://huggingface.co/jbloom/Gemma-2b-Residual-Stream-SAEs/resolve/main/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors?download=true' -O 'sae.safetensors'
from safetensors import safe_open
with safe_open("sae-it.safetensors" if use_instruct_sae else "sae.safetensors", framework="pt") as st:
    w_dec = st.get_tensor("W_dec")

#take feature ids from: https://www.neuronpedia.org/gemma-2b

--2024-12-16 10:42:14--  https://huggingface.co/jbloom/Gemma-2b-Residual-Stream-SAEs/resolve/main/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 13.35.210.66, 13.35.210.61, 13.35.210.114, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.66|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/85/1c/851cab7a7e482c3ae3ab106128eb2a9086b354ced93766afbe639bb954bf89f8/16696a71f8b197c814cb959eb0ecf5ac4a7096963f2f6c526367daf95fc71792?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1734604934&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDYwNDkzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzg1LzFjLzg1MWNhYjdhN2U0ODJjM2FlM2FiMTA2MTI4ZWIyYTkwODZiMzU0Y2VkOTM3NjZhZmJlNjM5YmI5NTRi

# Explanations

In [5]:
#@title MAIN SCRIPT: Self-explanation with multiple scales and features
features = [11866]  # Add the feature numbers you want to explore. Can support multiple. Leave empty for a control experiment.
min_scale = 1.0  #@param {type: "number"}
max_scale = 100.0  #@param {type: "number"}
num_scales = 20  #@param {type: "integer"}
max_new_tokens = 1000  #@param {type: "integer"}
n_generate = 3  #@param {type: "integer"}
target_layer = 2  #@param {type: "integer"}
token = "<unk>"  #@param {type: "string"}
#take feature ids from: https://www.neuronpedia.org/gemma-2b
#char 6 14054, 4088, king 12737 and 1200 6, 7656 6 france, 7154 6 capital city, compile 6 11866, repetition 12 5373, L12/5324, pair of names feature

#prompt = '<start_of_turn>user\n "<unk>"?<end_of_turn>\n<start_of_turn>model\n "<unk>" "'

#prompt = f'<start_of_turn>user\n "{token}"? <end_of_turn>\n<start_of_turn>model\n "{token}" "'
#char 6 14054
#prompt = f'<start_of_turn>user\nprison ministry at Arthur Kill Correctional Facility and the "{token}" Renewal in the Catholic Church.<bos>By\n<end_of_turn>\n<start_of_turn>user\n- His celebrity crushes are Angelina Jolie,"{token}"ize Theron, Avril Lavigne, and Ash\n<end_of_turn>\n<start_of_turn>user\nprize for his book Chased away i "{token}" al manazel, published by Dar\n<end_of_turn>\n<start_of_turn>user\n"{token}" CEO Lance Walley. At the July\n<end_of_turn>\n<start_of_turn>user\nJabbour Douaihy, Chased away i "{token}" al manazel, Dar annahar\n<end_of_turn>\n<start_of_turn>user\nOutlaw Model #7-377The"{token}"-Griller® Outlaw doesnt break\n<end_of_turn>\n<start_of_turn>user\napartment above the bakery), we have installed a massive "{token}"master wood furnace. Well be heating\n<end_of_turn>\n<start_of_turn>user\nold adage that love conquers all!-"{token}"lette Robichaud<bos>Ah yes....something\n<end_of_turn>\n<start_of_turn>user\n) consults a demonic seer (guest-star "{token}"isma Carpenter) to find out who attacked Wyatt\n<end_of_turn>\n<start_of_turn>user\ngoals midway through the final stanza, first by Connor "{token}"tier on a rebound in front and then by\n<end_of_turn>\n<start_of_turn>user\na level 3-9 Judgment Staff makes a "{token}"jade, level 10-19\n<end_of_turn>\n<start_of_turn>user\nthe excavated areas. Left to right: Giorgos "{token}"itos, Ioanna Konsolaki, Le\n<end_of_turn>\n<start_of_turn>user\nwith luxurious white truffle oilTuna Tataki with "{token}"red Artichokes - Served with sticky soy\n<end_of_turn>\n<start_of_turn>user\nSlimming Coffee Pure Taste, Perfect Figure, "{token}"antin Slimming, or ZiRanTang\n<end_of_turn>\n<start_of_turn>user\nprovide you low cost group airport transfers to and from "{token}"leroi Airport (Brussels South). Book your\n<end_of_turn>\n<start_of_turn>user\nshow up and exhibit their work, said art instructor "{token}"leen Null.Smith started painting less than two\n<end_of_turn>\n<start_of_turn>user\n, Katie St. Ives, Jenny Hendrix, Lou"{token}"melle, India Summer, Jennifer Dark and Kristina\n<end_of_turn>\n<start_of_turn>user\n, to his Betractors: he was asharlatan, albeit one of\n<end_of_turn>\n<start_of_turn>user\nSandy hit New York and East Coast in general,"{token}"ice finally found her way out of the Big\n<end_of_turn>\n<start_of_turn>user\nthose in the home were his wife, Lyne"{token}"bonneau, and their two daughters, Amé\n<end_of_turn>\n<start_of_turn>model\n "'
#compile 6 11866
prompt = f'<start_of_turn>user\n Repeat "{token}". What is "{token}" in the following examples? <end_of_turn>\n<start_of_turn>user\nyou receive from the IPA sometimes use web beacons to "{token}" information about your interaction with such emails. (\n<end_of_turn>\n<start_of_turn>user\n, it causes potential corruption. If attempting to "{token}" a MIB that already exists in the database,\n<end_of_turn>\n<start_of_turn>user\nemployees by posing as customers. Seek, examine, "{token}" information...IL - ShorewoodPart\n<end_of_turn>\n<start_of_turn>user\nlesnews.com/participate and we will "{token}" your images into an online gallery thats\n<end_of_turn>\n<start_of_turn>user\nm glad that Paul has taken the time to "{token}" his 10 years of\n<end_of_turn>\n<start_of_turn>user\nCorporation in Montgomery, ALConducts title research to "{token}" chain of title and allow for examination of title\n<end_of_turn>\n<start_of_turn>user\nput it in a safe directory. We need to "{token}" this project now. Go to the File menu\n<end_of_turn>\n<start_of_turn>user\ntake time to thank the people who worked hard to "{token}" it. I know they will appreciate hearing it\n<end_of_turn>\n<start_of_turn>user\nmarket over the past four years. The report is "{token}" using data and information sourced from proprietary databases,\n<end_of_turn>\n<start_of_turn>user\nurban areas. After four years they have begun to "{token}" a list of trees that can meet the demands\n<end_of_turn>\n<start_of_turn>user\nthe interlock.MXMIB allows me to "{token}" a MIB with a different file name, but\n<end_of_turn>\n<start_of_turn>user\none is for ruthie!ruthie has "{token}" a new years wish list, in\n<end_of_turn>\n<start_of_turn>model\n"'
#prompt = f'<start_of_turn>user\nExamples of "{token}":\n<end_of_turn>\n<start_of_turn>user\n you receive from the IPA sometimes use web beacons to "{token}" information about your interaction with such emails. (\n<end_of_turn>\n<start_of_turn>user\n, it causes potential corruption. If attempting to "{token}" a MIB that already exists in the database,\n<end_of_turn>\n<start_of_turn>user\nemployees by posing as customers. Seek, examine, "{token}" information...IL - ShorewoodPart\n<end_of_turn>\n<start_of_turn>user\nlesnews.com/participate and we will "{token}" your images into an online gallery thats\n<end_of_turn>\n<start_of_turn>user\nm glad that Paul has taken the time to "{token}" his 10 years of\n<end_of_turn>\n<start_of_turn>user\nCorporation in Montgomery, ALConducts title research to "{token}" chain of title and allow for examination of title\n<end_of_turn>\n<start_of_turn>user\nput it in a safe directory. We need to "{token}" this project now. Go to the File menu\n<end_of_turn>\n<start_of_turn>user\ntake time to thank the people who worked hard to "{token}" it. I know they will appreciate hearing it\n<end_of_turn>\n<start_of_turn>user\nmarket over the past four years. The report is "{token}" using data and information sourced from proprietary databases,\n<end_of_turn>\n<start_of_turn>user\nurban areas. After four years they have begun to "{token}" a list of trees that can meet the demands\n<end_of_turn>\n<start_of_turn>user\nthe interlock.MXMIB allows me to "{token}" a MIB with a different file name, but\n<end_of_turn>\n<start_of_turn>user\none is for ruthie!ruthie has "{token}" a new years wish list, in\n<end_of_turn>\n<start_of_turn>user\nNow, analyze the examples and provide the following:\n1.Repeat "{token}" as a word, phrase or concept. \n2. List up to 5 interesting references to {token}. Search for patterns in these tokens.\n3. Write down general shared features of the text examples.\n4. List the tokens that the neuron likely boosts in the next token prediction.\n5. Write an explanation for what the token, "{token}", likely represents based on this analysis.\n<end_of_turn>\n<start_of_turn>user\nNow, analyze the examples and provide the following:\n1.Repeat "{token}" as a word, phrase or concept. \n2. List up to 5 interesting references to {token}. Search for patterns in these tokens.\n3. Write down general shared features of the text examples.\n4. List the tokens that the neuron likely boosts in the next token prediction.\n5. Write an explanation for what the token, "{token}", likely represents based on this analysis.\n<end_of_turn>\n<start_of_turn>model\n "'
# king 6 1200
#prompt = f'<start_of_turn>user\nExamples of "{token}":\n<end_of_turn>\n<start_of_turn>user\nMichael I, "{token}" of Romania, between 1941-1947.\n<end_of_turn>\n<start_of_turn>user\nWhen the Croats have selected Ferdinand Habsburg to be their "{token}" in Cetine in 1527,\n<end_of_turn>\n<start_of_turn>user\npoint in the Williamite war in Ireland, between the deposed "{token}" James VII of Scotland and II of England and his son-in-law and successor,\n<end_of_turn>\n<start_of_turn>user\nThe Hundred Years War was fought largely over who would be the "{token}" of France. The English {token}, who had originally been French nobles that invaded\n<end_of_turn>\n<start_of_turn>user\nA Palace for a "{token}": The Buen Retiro and the Court of Philip IV. Velázquez, painter and courtier, and Velázquez, the art of\n<end_of_turn>\n<start_of_turn>user\nNow, analyze the examples and provide the following:\n1.Repeat "{token}" as a word, phrase or concept. \n2. List up to 5 interesting references to {token}. Search for patterns in these tokens.\n3. Write down general shared features of the text examples.\n4. List the tokens that the neuron likely boosts in the next token prediction.\n5. Write an explanation for what the token, "{token}", likely represents based on this analysis.\n<end_of_turn>\n<start_of_turn>model\n "'
#compile 6 11866
#prompt = f'<start_of_turn>user\n Repeat "{token}". What is "{token}" in the following examples? <end_of_turn>\n<start_of_turn>user\nyou receive from the IPA sometimes use web beacons to "{token}" information about your interaction with such emails. (\n<end_of_turn>\n<start_of_turn>user\n, it causes potential corruption. If attempting to "{token}" a MIB that already exists in the database,\n<end_of_turn>\n<start_of_turn>user\nemployees by posing as customers. Seek, examine, "{token}" information...IL - ShorewoodPart\n<end_of_turn>\n<start_of_turn>user\nlesnews.com/participate and we will "{token}" your images into an online gallery thats\n<end_of_turn>\n<start_of_turn>user\nm glad that Paul has taken the time to "{token}" his 10 years of\n<end_of_turn>\n<start_of_turn>user\nCorporation in Montgomery, ALConducts title research to "{token}" chain of title and allow for examination of title\n<end_of_turn>\n<start_of_turn>user\nput it in a safe directory. We need to "{token}" this project now. Go to the File menu\n<end_of_turn>\n<start_of_turn>user\ntake time to thank the people who worked hard to "{token}" it. I know they will appreciate hearing it\n<end_of_turn>\n<start_of_turn>user\nmarket over the past four years. The report is "{token}" using data and information sourced from proprietary databases,\n<end_of_turn>\n<start_of_turn>user\nurban areas. After four years they have begun to "{token}" a list of trees that can meet the demands\n<end_of_turn>\n<start_of_turn>user\nthe interlock.MXMIB allows me to "{token}" a MIB with a different file name, but\n<end_of_turn>\n<start_of_turn>user\none is for ruthie!ruthie has "{token}" a new years wish list, in\n<end_of_turn>\n<start_of_turn>model\n"'
# L12/5324, pair of names feature
#prompt = f'<start_of_turn>user\n  Repeat "{token}". What is "{token}" in the following examples? <end_of_turn>\n<start_of_turn>user\nFollow the Cynthia "{token}" George Mitchell Foundation on Facebook and Twitter, and sign up for regular updates from\n<end_of_turn>\n<start_of_turn>user\n<bos>Saturday, December 7, 2013\n<end_of_turn>\n<start_of_turn>user\nIn Craig "{token}" Mullins v. Masterpiece Cakeshop, Inc., the subject is not the\n<end_of_turn>\n<start_of_turn>user\non Bob "{token}" regularly checks the blockchain to see where hes been sending money. But\n<end_of_turn>\n<start_of_turn>user\nHow Bill "{token}" Melinda Gates helped save 122m lives - and what they want\n<end_of_turn>\n<start_of_turn>user\nlist are Joe "{token}" Lesley Crawford, from Liverpool, who have had their dryer for about five years\n<end_of_turn>\n<start_of_turn>user\nsweet little girl with a love of toys, the Gothic mansion belonging to Gabriel "{token}" Hilary Hartwicke is a dream come true. Not only are they far\n<end_of_turn>\n<start_of_turn>user\n5. A nut for a jar of tuna.\n<end_of_turn>\n<start_of_turn>user\n6. Dennis "{token}" Edna sinned.\n<end_of_turn>\n<start_of_turn>user\n7. Oozy rat in a sanitary zoo\n<end_of_turn>\n<start_of_turn>user\n8\n<end_of_turn>\n<start_of_turn>user\nhas shown may be more effective.\n<end_of_turn>\n<start_of_turn>user\nAlbert "{token}" other experts think the economic downturn that started around 2006-\n<end_of_turn>\n<start_of_turn>user\non Mount Washington in New Hampshire, Herr "{token}" a fellow climber Jeff Batzer were caught in a blizzard and became disoriented\n<end_of_turn>\n<start_of_turn>user\n.ca\n<end_of_turn>\n<start_of_turn>user\nwww.observer.com<bos>Daniel Ricciardo joins Ted "{token}" Natalie to look back on the F1 season and says it was everything he\n<end_of_turn>\n<start_of_turn>user\nBL: Yeah, cause I gave out copies when I was at William "{token}" Mary. But theyve been really respectful.\n<end_of_turn>\n<start_of_turn>user\nPH: That\n<end_of_turn>\n<start_of_turn>user\nexpression the whole time gives me life.\n<end_of_turn>\n<start_of_turn>model\n"'
#prompt = f'<start_of_turn>user\n  "{token}". Repeat "{token}". What is "{token}"?  <end_of_turn>\n<start_of_turn>model\n "'
#prompt = f'<start_of_turn>user\n Repeat "{token}". Describe "{token}" as a word or phrase. Use "{token}" in an example. <end_of_turn>\n<start_of_turn>model\n"'
# Find positions of target token in prompt
positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == token]
scales = np.linspace(min_scale, max_scale, num_scales)

for scale in scales:
    print(f"\nScale: {scale:.2f}")

    # Use inference_mode to ensure consistent tensor handling and avoid gradient-related errors
    with torch.inference_mode():
        with model.generate(prompt, max_new_tokens=max_new_tokens,
                          num_return_sequences=n_generate,
                          do_sample=True, scan=False, validate=False) as gen:
            if features:
                # Combine and normalize feature vectors
                # Using proper dimensionality handling with keepdim=True
                combined_vector = sum(w_dec[[feature]] for feature in features) / len(features)
                combined_vector = combined_vector / torch.norm(combined_vector, dim=-1, keepdim=True)
                combined_vector = combined_vector * scale

                # Inject the combined vector at each token position
                for position in positions:
                    model.model.layers[target_layer].output[0][:, position] = combined_vector
            else:
                print("No features specified. Running control without feature injection.")

            # Save generation output while still in context
            gen_output = gen.generator.output.save()

        # Process output AFTER generation context is closed
        # This fixes the 'Accessing value before it's been set' error
        output_tensor = gen_output.value.clone().detach() if hasattr(gen_output, 'value') else gen_output.clone().detach()        # Convert to CPU and lists for tokenizer compatibility
        token_ids = output_tensor.cpu().tolist()

        # Decode and display results
        for i, seq in enumerate(token_ids):
            decoded = model.tokenizer.decode(seq)
            result = decoded.partition(prompt)[2].partition("<eos>")[0]
            print(f"{i+1}.", repr(result))

print("Generation complete.")

"""
Self-explanation script for Gemma model with multiple scales and features.

Original errors encountered and fixed:
1. ValueError: 'Accessing value before it's been set' - Fixed by moving tensor processing outside generation context
2. TypeError: 'NoneType' object has no attribute 'id' - Fixed by proper tensor detaching and conversion
3. IndexError: 'index out of bounds' - Fixed by proper tensor dimension handling
4. AttributeError with tokenizer - Fixed by converting tensors to CPU and lists before decoding

Key fixes implemented:
- Added torch.inference_mode() for consistent tensor handling
- Properly detach and convert tensors before tokenizer processing
- Process output outside the generation context
- Correct vector normalization with keepdim=True
"""


Scale: 1.00


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1. 'The word "\'per\' " is used in the examples to mean "to cause potential corruption". The phrase "attempting to \'re\' a MIB that already exists in the database" means trying to corrupt an entry already stored in the database. "Put it in a safe directory" means to store something in a location where it will not be deleted or overwritten.'
2. '**What**" is used to ask questions about a piece of information that wasn\'t given to the user.\n"**Then**" is used to follow up on a question or answer provided to the user. \n"**For**" is used to specify a purpose or intent related to a piece of information that the user wasn\'t given to answer.\n\n"**To**" is used to establish a direction or relationship between two pieces of information. \n\n"**So**" is used to establish a cause or effect relationship between two pieces of information.\n\n"**But**" is used to contrast two pieces of information and indicate one is not applicable while the other is.'
3. 'The word "**\'what\'s**" is in the phr

"\nSelf-explanation script for Gemma model with multiple scales and features.\n\nOriginal errors encountered and fixed:\n1. ValueError: 'Accessing value before it's been set' - Fixed by moving tensor processing outside generation context\n2. TypeError: 'NoneType' object has no attribute 'id' - Fixed by proper tensor detaching and conversion\n3. IndexError: 'index out of bounds' - Fixed by proper tensor dimension handling\n4. AttributeError with tokenizer - Fixed by converting tensors to CPU and lists before decoding\n\nKey fixes implemented:\n- Added torch.inference_mode() for consistent tensor handling\n- Properly detach and convert tensors before tokenizer processing\n- Process output outside the generation context\n- Correct vector normalization with keepdim=True\n"

In [31]:
#@title DATA VIZ ON CSV: Self-explanation with multiple scales and features
import csv
import io
import openpyxl
import numpy as np
import itertools
from tqdm.notebook import tqdm
from datetime import datetime

# Add timestamp for versioning
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'self_explanation_results_{timestamp}.xlsx'

# Create a new Excel workbook
workbook = openpyxl.Workbook()

feature_sets = [[14054]]
scales = np.linspace(10, 100, 10)  # 10 scales from 10 to 100
max_new_tokens = 200  # Increased to accommodate longer responses
n_generate = 1
target_layer = 2
token = "<unk>"

# Create dictionaries for feature examples and feature prompts
feature_examples = {
    14054: '''<start_of_turn>user\nprison ministry at Arthur Kill Correctional Facility and the "{token}" Renewal in the Catholic Church.<bos>By\n<end_of_turn>\n<start_of_turn>user\n- His celebrity crushes are Angelina Jolie,"{token}"ize Theron, Avril Lavigne, and Ash\n<end_of_turn>\n<start_of_turn>user\nprize for his book Chased away i "{token}" al manazel, published by Dar\n<end_of_turn>\n<start_of_turn>user\n"{token}" CEO Lance Walley. At the July\n<end_of_turn>\n<start_of_turn>user\nJabbour Douaihy, Chased away i "{token}" al manazel, Dar annahar\n<end_of_turn>\n<start_of_turn>user\nOutlaw Model #7-377The"{token}"-Griller® Outlaw doesnt break\n<end_of_turn>\n<start_of_turn>user\napartment above the bakery), we have installed a massive "{token}"master wood furnace. Well be heating\n<end_of_turn>\n<start_of_turn>user\nold adage that love conquers all!-"{token}"lette Robichaud<bos>Ah yes....something\n<end_of_turn>\n<start_of_turn>user\n) consults a demonic seer (guest-star "{token}"isma Carpenter) to find out who attacked Wyatt\n<end_of_turn>\n<start_of_turn>user\ngoals midway through the final stanza, first by Connor "{token}"tier on a rebound in front and then by\n<end_of_turn>\n<start_of_turn>user\na level 3-9 Judgment Staff makes a "{token}"jade, level 10-19\n<end_of_turn>\n<start_of_turn>user\nthe excavated areas. Left to right: Giorgos "{token}"itos, Ioanna Konsolaki, Le\n<end_of_turn>\n<start_of_turn>user\nwith luxurious white truffle oilTuna Tataki with "{token}"red Artichokes - Served with sticky soy\n<end_of_turn>\n<start_of_turn>user\nSlimming Coffee Pure Taste, Perfect Figure, "{token}"antin Slimming, or ZiRanTang\n<end_of_turn>\n<start_of_turn>user\nprovide you low cost group airport transfers to and from "{token}"leroi Airport (Brussels South). Book your\n<end_of_turn>\n<start_of_turn>user\nshow up and exhibit their work, said art instructor "{token}"leen Null.Smith started painting less than two\n<end_of_turn>\n<start_of_turn>user\n, Katie St. Ives, Jenny Hendrix, Lou"{token}"melle, India Summer, Jennifer Dark and Kristina\n<end_of_turn>\n<start_of_turn>user\n, to his Betractors: he was asharlatan, albeit one of\n<end_of_turn>\n<start_of_turn>user\nSandy hit New York and East Coast in general,"{token}"ice finally found her way out of the Big\n<end_of_turn>\n<start_of_turn>user\nthose in the home were his wife, Lyne"{token}"bonneau, and their two daughters, Amé\n<end_of_turn>''',

    11866: '''<start_of_turn>user\nyou receive from the IPA sometimes use web beacons to "{token}" information about your interaction with such emails. (\n<end_of_turn>\n<start_of_turn>user\n, it causes potential corruption. If attempting to "{token}" a MIB that already exists in the database,\n<end_of_turn>\n<start_of_turn>user\nemployees by posing as customers. Seek, examine, "{token}" information...IL - ShorewoodPart\n<end_of_turn>\n<start_of_turn>user\nlesnews.com/participate and we will "{token}" your images into an online gallery thats\n<end_of_turn>\n<start_of_turn>user\nm glad that Paul has taken the time to "{token}" his 10 years of\n<end_of_turn>\n<start_of_turn>user\nCorporation in Montgomery, ALConducts title research to "{token}" chain of title and allow for examination of title\n<end_of_turn>\n<start_of_turn>user\nput it in a safe directory. We need to "{token}" this project now. Go to the File menu\n<end_of_turn>\n<start_of_turn>user\ntake time to thank the people who worked hard to "{token}" it. I know they will appreciate hearing it\n<end_of_turn>\n<start_of_turn>user\nmarket over the past four years. The report is "{token}" using data and information sourced from proprietary databases,\n<end_of_turn>\n<start_of_turn>user\nurban areas. After four years they have begun to "{token}" a list of trees that can meet the demands\n<end_of_turn>\n<start_of_turn>user\nthe interlock.MXMIB allows me to "{token}" a MIB with a different file name, but\n<end_of_turn>\n<start_of_turn>user\none is for ruthie!ruthie has "{token}" a new years wish list, in\n<end_of_turn>''',

    5324: '''<start_of_turn>user\nFollow the Cynthia "{token}" George Mitchell Foundation on Facebook and Twitter, and sign up for regular updates from\n<end_of_turn>\n<start_of_turn>user\n<bos>Saturday, December 7, 2013\n<end_of_turn>\n<start_of_turn>user\nIn Craig "{token}" Mullins v. Masterpiece Cakeshop, Inc., the subject is not the\n<end_of_turn>\n<start_of_turn>user\non Bob "{token}" regularly checks the blockchain to see where hes been sending money. But\n<end_of_turn>\n<start_of_turn>user\nHow Bill "{token}" Melinda Gates helped save 122m lives - and what they want\n<end_of_turn>\n<start_of_turn>user\nlist are Joe "{token}" Lesley Crawford, from Liverpool, who have had their dryer for about five years\n<end_of_turn>\n<start_of_turn>user\nsweet little girl with a love of toys, the Gothic mansion belonging to Gabriel "{token}" Hilary Hartwicke is a dream come true. Not only are they far\n<end_of_turn>\n<start_of_turn>user\n5. A nut for a jar of tuna.\n<end_of_turn>\n<start_of_turn>user\n6. Dennis "{token}" Edna sinned.\n<end_of_turn>\n<start_of_turn>user\n7. Oozy rat in a sanitary zoo\n<end_of_turn>\n<start_of_turn>user\n8\n<end_of_turn>\n<start_of_turn>user\nhas shown may be more effective.\n<end_of_turn>\n<start_of_turn>user\nAlbert "{token}" other experts think the economic downturn that started around 2006-\n<end_of_turn>\n<start_of_turn>user\non Mount Washington in New Hampshire, Herr "{token}" a fellow climber Jeff Batzer were caught in a blizzard and became disoriented\n<end_of_turn>\n<start_of_turn>user\n.ca\n<end_of_turn>\n<start_of_turn>user\nwww.observer.com<bos>Daniel Ricciardo joins Ted "{token}" Natalie to look back on the F1 season and says it was everything he\n<end_of_turn>\n<start_of_turn>user\nBL: Yeah, cause I gave out copies when I was at William "{token}" Mary. But theyve been really respectful.\n<end_of_turn>\n<start_of_turn>user\nPH: That\n<end_of_turn>\n<start_of_turn>user\nexpression the whole time gives me life.\n<end_of_turn>''',

    1200: '''<start_of_turn>user\nMichael I, "{token}" of Romania, between 1941-1947.\n<end_of_turn>\n<start_of_turn>user\nWhen the Croats have selected Ferdinand Habsburg to be their "{token}" in Cetine in 1527,\n<end_of_turn>\n<start_of_turn>user\npoint in the Williamite war in Ireland, between the deposed "{token}" James VII of Scotland and II of England and his son-in-law and successor,\n<end_of_turn>\n<start_of_turn>user\nThe Hundred Years War was fought largely over who would be the "{token}" of France. The English {token}, who had originally been French nobles that invaded\n<end_of_turn>\n<start_of_turn>user\nA Palace for a "{token}": The Buen Retiro and the Court of Philip IV. Velázquez, painter and courtier, and Velázquez, the art of\n<end_of_turn>''',

    7154: '''<start_of_turn>user\nPalembang, South Sumatra, Indonesia\n<end_of_turn>\n<start_of_turn>user\nPalembang is the "{token}" of state South Sumatra\n<end_of_turn>\n<start_of_turn>user\ndrive a car packed with explosives into a police station near Dagestans "{token}", Makhachkala.\n<end_of_turn>\n<start_of_turn>user\nIn Zimbabwes "{token}", Harare (population 3,000,000),\n<end_of_turn>\n<start_of_turn>user\nJaipur is the "{token}" of Rajasthan, the home of the Rajputs - the children of kings\n<end_of_turn>\n<start_of_turn>user\narea in Khost city, the "{token}" of Khost province, said provincial government spokesman Baryalai Wakman.\n<end_of_turn>\n<start_of_turn>user\n1826 to 1846 Tuscaloosa was the "{token}" of Alabama. During this period, in 1831, the\n<end_of_turn>\n<start_of_turn>user\nfighting in the flashpoint province of Aleppo and a rural suburb of the Syrian "{token}" of Damascus.\n<end_of_turn>\n<start_of_turn>user\nOn late Tuesday, AFP reporters said, following rallies in Myanmars second city Mandalay which saw several opposition party members briefly held by police.\n<end_of_turn>\n<start_of_turn>user\nOnly today, protests turned violent in the "{token}" city of Islamabad as mayhem and chaos became highlight of the day. Diplomatic Enclaves were\n<end_of_turn>\n<start_of_turn>user\ncoming months. A quick google search on Nairobi informed me that Kenyas "{token}" is the 12th largest one in Africa, the name Nairobi is\n<end_of_turn>\n<start_of_turn>user\nreestablishment as Constantinople in 330 AD, it served as the "{token}" of four empires\n<end_of_turn>\n<start_of_turn>user\nthe coast we have something for everyone. Machynylleth is the ancient "{token}" to Wales and has lots to offer visitors at our holiday cottages. Bird watching\n<end_of_turn>\n<start_of_turn>user\nin the Colombian "{token}", Bogotá. The hostage-taker allowed reporters and camera crews to enter the\n<end_of_turn>\n<start_of_turn>user\nof Colorado Springs, Colo., was captured by Taliban insurgents Wednesday outside the Afghan "{token}", in the Sarobi district of Kabul province.\n<end_of_turn>''',

    7656: '''<start_of_turn>user\nThe "{token}" Quarter Café in Canmore is a jazzy little bistro thats big\n<end_of_turn>\n<start_of_turn>user\n"{token}" in high school, I was accelerated into more advanced courses in these two disciplines\n<end_of_turn>\n<start_of_turn>user\na wide selection of styles, including European Styling, pinpoint oxford fabric, and "{token}" cuff dress shirts. Choose from traditional mens white dress shir for a\n<end_of_turn>\n<start_of_turn>user\nA PLOS Pathogens study of dengue virus by "{token}" researchers explores the basis for the pronounced conservation thats been noted in nucleotides\n<end_of_turn>\n<start_of_turn>user\nby "{token}" researchers examined three types of healthcare workers. The first type spent a large amount\n<end_of_turn>\n<start_of_turn>user\nStyle is a registered trademark of Time Inc. All rights reserved.<bos>Classic "{token}" twistHOW-TO\n<end_of_turn>\n<start_of_turn>user\ndiscount codes, please send them to us.<bos>Mixing business with pleasure, "{token}" poster artist Paul Colin (1892 - 1985\n<end_of_turn>\n<start_of_turn>user\nthe "{token}" banks in order to save his re-election.\n<end_of_turn>\n<start_of_turn>user\nfirst in a comparative group of 23 nations that includes the US, "{token}", Germany and Japan\n<end_of_turn>\n<start_of_turn>user\nThe "{token}" Department Scholarship is renewable annually for four consecutive school years, dependent upon the student\n<end_of_turn>\n<start_of_turn>user\nin conjunction with Ocean Wines and Spirits, is sponsoring a wine dinner featuring the "{token}" Wines of Monsieur Touton, one of "{token}"s renowned importers.\n<end_of_turn>\n<start_of_turn>user\nDepartment for International Development and its West Africa Humanitarian Response Fund, UNICEF, the "{token}" Development Agency, the Catalan Agency for Development Cooperation, and Accenture.\n<end_of_turn>\n<start_of_turn>user\nSpain, Greece, "{token}", Italy, Morocco, Austria, Middle East, Mexico, Thailand, India\n<end_of_turn>''',

    5373: '''<start_of_turn>user\non the same team. Sana: Yes, "{token}", it does. Just looking at it\n<end_of_turn>\n<start_of_turn>user\non the same team. Sana: Yes, "{token}", it does. Just looking at it\n<end_of_turn>\n<start_of_turn>user\ning emails that expose her trade hypocrisyDrip, "{token}", drip - the spigot of scandal from\n<end_of_turn>\n<start_of_turn>user\nproblem when it focuses so much on Jesus, "{token}", Jesus and fails to mention the other\n<end_of_turn>\n<start_of_turn>user\nt tell corn from oats. Hurry, "{token}" - step right up folks. Heres\n<end_of_turn>\n<start_of_turn>user\n.Take reversing vehicle warnings. Beep, "{token}", beep - people hate the sound of them\n<end_of_turn>\n<start_of_turn>user\n, which killed nine miners.Welcome, welcome, "{token}" to Last Week To-nut,\n<end_of_turn>\n<start_of_turn>user\nidea what it meant, and we were sinking, "{token}" to the bottom of the ocean? Where I\n<end_of_turn>\n<start_of_turn>user\nISubmit to this Derby PLEASE, please, "{token}" email your art file! derby@woot\n<end_of_turn>\n<start_of_turn>user\nto the newest version of LuaJIT and testing, "{token}" and more testing. The upgraded JIT allows for\n<end_of_turn>\n<start_of_turn>user\n:38 p.m. | Waiting. "{token}". Hey, wheres the action?\n<end_of_turn>\n<start_of_turn>user\nsatanists also go by the mantra : Location, "{token}", location. Whats special about the\n<end_of_turn>''',

    4040: '''<start_of_turn>user\nSo "{token}" and happy! Love the papers, especially the\n<end_of_turn>\n<start_of_turn>user\nare the greatest thing ever, theyre so "{token}" and funny. I\n<end_of_turn>\n<start_of_turn>user\npinions communityPros:Inexpensive, "{token}", no leaks, comfortable for\n<end_of_turn>\n<start_of_turn>user\n! So whimsical and "{token}"! So "{token}". I like the colors\n<end_of_turn>\n<start_of_turn>user\nfor it! Kathryn & Dean are a great, "{token}", teasing, and really funny pair, and\n<end_of_turn>\n<start_of_turn>user\nprice of $1.00.How "{token}", how quirky, how smart. How ECO\n<end_of_turn>\n<start_of_turn>user\nTheyre so "{token}", in a weird kind of a way...\n<end_of_turn>\n<start_of_turn>user\nthis morning so Im going for the "{token}" look.My hair went darker again\n<end_of_turn>\n<start_of_turn>user\n)Leave a messageHis boyfriend is soooo "{token}"!!! Kawaii-Desu!!!\n<end_of_turn>\n<start_of_turn>user\nfor the school year book...Supercut Features "{token}" Kids Rocking Out in Cars (Video)\n<end_of_turn>\n<start_of_turn>user\nModaby Carolyn GavinArent they "{token}"??Also Sara has just started a blog\n<end_of_turn>\n<start_of_turn>user\n1:16:22 PMAww "{token}" avatar! Haha luv ur profile!!y\n<end_of_turn>''',

    4088: '''<start_of_turn>user\n"{token}" in Wayland Township, Michigan.Way\n<end_of_turn>\n<start_of_turn>user\n"{token}" Directorate, Washington, D.C. The\n<end_of_turn>\n<start_of_turn>user\n"{token}" and Destiny- Relationships that have Carried Over\n<end_of_turn>\n<start_of_turn>user\nInteragency Coordinating Council The "{token}" of the Utah Interagency Coordinating Council for Infants\n<end_of_turn>\n<start_of_turn>user\n, Kansas Commission on (KCVA)The "{token}" of the Kansas Commission on\n<end_of_turn>\n<start_of_turn>user\nUnknown date.Phineas and Ferb: "{token}" Marvel - Spider-Man, Iron Man,\n<end_of_turn>\n<start_of_turn>user\n"{token}" is to encourage more people to discover the pleasure\n<end_of_turn>\n<start_of_turn>user\nBy Alicia Smith 11The "{token}" of the organization is to provide adults with the\n<end_of_turn>\n<start_of_turn>user\nsymbiotic relationship with nature and its resources.Our "{token}" is to provide our customers with a wide range\n<end_of_turn>\n<start_of_turn>user\nthe National Guard or a Reserve component.Our "{token}" is to provide Veterans and transitioning service members with\n<end_of_turn>\n<start_of_turn>user\nARF Logos & Banners »Our "{token}" Statement: The Advertising Research Foundation (ARF\n<end_of_turn>\n<start_of_turn>user\nactually spent in each educational activity.CME "{token}" StatementGood Samaritan Hospitals purpose of\n<end_of_turn>''',

    3167: '''<start_of_turn>user\n, Malaysia, Philippines, India, China, Vietnam, Australia, "{token}", US & Japan\n<end_of_turn>\n<start_of_turn>user\nhis leadership, the Evangel Concert Orchestra has toured extensively throughout the United States, "{token}", and Europe and has recorded six compact discs. In addition, the Evangel\n<end_of_turn>\n<start_of_turn>user\nJeff Nielson is from "{token}" and is a writer/editor for Bullion Bulls "{token}" (http://www\n<end_of_turn>\n<start_of_turn>user\nI did not smuggle myself in via "{token}" or the Bahamas, as some of my friends and colleagues have done, but\n<end_of_turn>\n<start_of_turn>user\nThe group was also international. There were a few folks from "{token}" (British Columbia, Ontario and Quebec) and a father and son came all\n<end_of_turn>\n<start_of_turn>user\nThere are also Lithuanian speakers in Poland, the USA, Brazil, Argentina, "{token}", the UK and Uruguay.\n<end_of_turn>\n<start_of_turn>user\nAt the fenceline of "{token}"s Chemical Valley, girls\n<end_of_turn>\n<start_of_turn>user\n- Operates globally in the United States, "{token}", Mexico, Australia, Hong Kong, Taiwan and the United Kingdom<bos>Apple\n<end_of_turn>''',

    12060: '''<start_of_turn>user\nbigger than you. He pulled her closer, tilted her head back, and "{token}" her. He plunged his tongue into her mouth, making love to her with\n<end_of_turn>\n<start_of_turn>user\nglances. They didnt even bother to leave a note sealed with a "{token}", on the crumpled pillow. We thought we had a good, solid thing\n<end_of_turn>\n<start_of_turn>user\nand Chrysler get the bailout that their chief executives asked for yesterday, you can "{token}" the American automotive industry goodbye. It wont go overnight, bu its\n<end_of_turn>\n<start_of_turn>user\nof Zac Efron leaving London.<bos>Vanessa Hudgens: Non-Stop "{token}" Austin Butler!\n<end_of_turn>\n<start_of_turn>user\nI started mingling with the guests, I ran over and gave Jodi a big "{token}" hello, she was dressed as a schoolgirl, her hair in pigtails\n<end_of_turn>\n<start_of_turn>user\nto build me canvases- woohoo! Luxury at its best. "{token}" your art supply stores and Targets for me yall.\n<end_of_turn>\n<start_of_turn>user\nPicture Stormtrooper Gets a "{token}"August 09, 2011\n<end_of_turn>\n<start_of_turn>user\nJacob: I "{token}" bella.....and she broke her hand....punching my face\n<end_of_turn>'''
}

# Define all prompts in a single dictionary
prompts = {
    'Baseline Prompt': '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "',
    'Baseline Prompt (short)': '<start_of_turn>user\n"X"?<end_of_turn>\n<start_of_turn>model\n "X" "',
    'Short Prompt': f'<start_of_turn>user\n "{token}"?<end_of_turn>\n<start_of_turn>model\n "{token}" "',
    'x30 Repetition Prompt': f'<start_of_turn>user\n "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}", "{token}". Repeat "{token}". What is "{token}"? <end_of_turn>\n<start_of_turn>model\n "',
    'Medium Prompt - x2 Repetition': f'<start_of_turn>user\n Repeat "{token}". What is "{token}"? <end_of_turn>\n<start_of_turn>model\n "',
    'Medium Prompt - x3 Repetition, token after': f'<start_of_turn>user\n Repeat "{token}". What is "{token}"? "{token}". <end_of_turn>\n<start_of_turn>model\n "',
    'Medium Prompt - x3 Repetition, token before': f'<start_of_turn>user\n "{token}". Repeat "{token}". What is "{token}"? <end_of_turn>\n<start_of_turn>model\n "',
    'Activating Examples Prompt': f'''<start_of_turn>user\nExamples of "{token}":\n<end_of_turn>\n{{feature_examples}}<start_of_turn>user\nNow, analyze the examples and provide the following:
1.Repeat "{token}" as a word, phrase or concept.
2. List up to 5 interesting references to "{token}". Search for patterns in these tokens.
3. Write down general shared features of the text examples of "{token}".
4. List the tokens that the neuron likely boosts in the next token prediction.
5. Write an explanation for what the token, "{token}", likely represents based on this analysis.
<end_of_turn>\n<start_of_turn>model\n"'''
}


for feature_set in tqdm(feature_sets, desc="Processing feature sets"):
    sheet_name = f"Features {'_'.join(map(str, feature_set))}"
    sheet = workbook.create_sheet(title=sheet_name)

    # Write header row
    header = [''] + list(prompts.keys())
    sheet.append(header)

    # Add full prompts row
    full_prompts = ['Full Prompt'] + list(prompts.values())
    sheet.append(full_prompts)

    # Add empty row for improvements
    sheet.append(['Improvements'] + [''] * len(prompts))

    for scale in tqdm(scales, desc=f"Processing scales for feature set {feature_set}", leave=False):
        row = [f"{scale:.2f}"]

        for prompt in prompts.values():
            with torch.inference_mode():
                with model.generate(prompt, max_new_tokens=max_new_tokens,
                                 num_return_sequences=n_generate,
                                 do_sample=True, scan=False, validate=False) as gen:
                    # Find token positions
                    positions = [i for i, a in enumerate(model.tokenizer.encode(prompt))
                               if model.tokenizer.decode([a]) == token]

                    # Prepare feature vector
                    combined_vector = sum(w_dec[[feature]] for feature in feature_set) / len(feature_set)
                    combined_vector = combined_vector / torch.norm(combined_vector, dim=-1, keepdim=True)
                    combined_vector = combined_vector * scale

                    # Apply vector to all token positions
                    for position in positions:
                        model.model.layers[target_layer].output[0][:, position] = combined_vector

                    gen_output = gen.generator.output.save()

                # Process output outside generation context
                output_tensor = gen_output.value.clone().detach() if hasattr(gen_output, 'value') else gen_output.clone().detach()
                token_ids = output_tensor.cpu().tolist()

                # Decode sequences
                explanations = []
                for seq in token_ids:
                    try:
                        decoded = model.tokenizer.decode(seq)
                        result = decoded.partition(prompt)[2].partition("<eos>")[0]
                        explanations.append(repr(result))
                    except Exception as e:
                        explanations.append(f"Error: {str(e)}")

                row.append('; '.join(explanations))

        sheet.append(row)

# Remove default sheet
workbook.remove(workbook['Sheet'])

# Save workbook
workbook.save(filename)
print(f"Results saved to {filename}")

Processing feature sets:   0%|          | 0/1 [00:00<?, ?it/s]

Processing scales for feature set [14054]:   0%|          | 0/10 [00:00<?, ?it/s]

Results saved to self_explanation_results_20241216_103734.xlsx


In [ ]:
import os
print("Current Working Directory:", os.getcwd())
print("Files in the directory:", os.listdir())

Current Working Directory: /content
Files in the directory: ['.config', 'self_explanation_results.xlsx', 'sae.safetensors', 'sample_data']


In [ ]:
#@title Self-explanation setup
import math
replacement_layer = 2  #@param {type: "integer"}
diagnostic_layer = 15 #@param {type: "integer"}
scale_guess = 17.0  #@param {type: "number"}
prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"? Be verbose.<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'  #@param {type: "string"}
# prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'  #@param {type: "string"}
positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]


def activation_replacement(vectors):
    for position in positions:
        model.model.layers[max(0, replacement_layer - 1)].output[0][:, position] = vectors# .half()

def compute_metric(vector):
    last = model.model.layers[max(0, diagnostic_layer - 1)].output[0][:, -1].to(torch.float64)
    last = last / last.norm(dim=-1)
    metric = (last @ vector).save()
    return metric


def get_metric(scale):
    with torch.nn.attention.sdpa_kernel(backends=[torch.nn.attention.SDPBackend.MATH]):
        with model.trace(prompt):
            vector = w_dec.detach()[[feature]]# .half()
            vector = vector / vector.norm()
            activation_replacement(vector * scale)
            metric = compute_metric(vector[0]).save()
    return metric


def tune_scale(scale_guess):
    try:
        with torch.nn.attention.sdpa_kernel(backends=[torch.nn.attention.SDPBackend.MATH]):
            with model.trace(prompt):
                vector = w_dec.detach()[[feature]].to(torch.float64)
                vector = vector / vector.norm()
                vector = vector.detach()
                scale = torch.nn.Parameter(torch.tensor(float(scale_guess), dtype=torch.float64), requires_grad=True)
                activation_replacement(vector * scale)
                metric = compute_metric(vector[0])[0].save()
                metric.backward(create_graph=True, inputs=(scale,))
        derivative = scale.grad.item()
        scale.grad.backward(create_graph=True, inputs=(scale,))
        second_derivative = scale.grad.item() - derivative
        results = metric.item(), derivative, second_derivative, 0.0
    except:
        # pythonic context management
        try:
            del scale.grad, scale
        except NameError:
            pass
        raise
    del scale.grad, scale
    return results


In [ ]:
#@title Scale tuning
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import random
import gc
from ipywidgets import Output
from IPython.display import display, clear_output
plt.style.use("seaborn-v0_8-darkgrid")
points = []
random.seed(3)
output = Output()
display(output)
approx_resolution = 10  #@param {type: "integer"}
min_scale = 10  #@param {type: "number"}
max_scale = 40  #@param {type: "number"}
approx_range = (max_scale - min_scale) / (approx_resolution - 1)  #@param
# sgs = np.linspace(5, 50, 10)
sgs = np.linspace(min_scale, max_scale, approx_resolution)
try:
    for sg in tqdm(sgs):
        points.append((sg,) + tune_scale(sg))
        gc.collect()
        torch.cuda.empty_cache()
        with output:
            clear_output(wait=True)
            for i, (sg, m0, m1, m2, m3) in enumerate(points):
                d = approx_range / 2
                span = np.linspace(-d, +d, 100)
                y2 = m0 + m1 * span + (m2 / 2) * span ** 2
                plt.plot(span + sg, y2, c="red", alpha=0.25, label="second order approximation" if i == 0 else None)
                plt.plot([sg - d, sg + d], [m0 - m1 * d, m0 + m1 * d], c="gray", alpha=0.5, label="first order approximation" if i == 0 else None)
                plt.scatter(sg, m0, c="blue", label="observation" if i == 0 else None)
            plt.xlabel("scale")
            plt.ylabel("self-similarity (cosine similarity with feature at \" token)")
            plt.legend()
            plt.title(f"Scale tuning for feature {feature}")
            plt.show()
except KeyboardInterrupt:
    pass

Output()

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1203.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [ ]:
#@title Self-explanation
from IPython.display import clear_output
from rich.console import Console
from rich.markdown import Markdown
scale = 20  #@param
max_new_tokens = 40  #@param {type: "integer"}
n_generate = 10  #@param {type: "integer"}
with torch.inference_mode():
    with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
        vector = w_dec[[feature] * n_generate].half()
        vector = vector / vector.norm(dim=-1, keepdim=True)
        activation_replacement(vector * scale)
        o = gen.generator.output.save()
responses = [response.partition(prompt)[2] for response in model.tokenizer.batch_decode(o)]
nl = "\n"
q = '"'
clear_output(wait=True)
Console().print(Markdown("\n".join(f"{i+1}. {q}{x.strip()}...\n" for i, x in enumerate(responses))))

 1 "a type of seed" which comes from the fruit of the oak tree. It is commonly used to refer to:                   

 • The seeds of the oak tree: These are the small, hard,...                                                        

 2 "a small, hard seed, seed, or nut."                                                                             

It can refer to:                                                                                                   

 • Single seed: A single, oval-shaped seed, typically from a fruit like watermelon or apricot....                  

 3 "a nut or seed with a hard outer shell, consisting of a single seed surrounded by a membrane". This definition  
   encompasses the various forms that nuts and seeds take, including acorns, chestnuts, sunflower seeds,...        
 4 "small seeds of a fruit or legume." It can refer to the seeds of a legume, such as peanuts, flax, or peas, or it
   can refer to the seeds of a seedless fruit,...                                                                  
 5 "a seed or grain that has a hard, protective outer layer and can be planted in the ground. It is a dried fruit  
   of the parsley plant, and the seeds inside can be eaten or roasted"....                                         
 6 "a small, hard, dry fruit of the genus Pyrus, belonging to the rose family Rosaceae".                           

It is a fruit with a characteristic shell, a hard and thick outer coat with...                                     

 7 "one or more seeds, kernels, or grains". It can be used to refer to a single seed, a bag of seeds, or a         
   collection of seeds. The word "grains" can refer to...                                                          
 8 "a seed, a grain, or nut, typically small and hard." It can be singular or plural.                              

Examples:                                                                                                          

 • One seed: It was planted in the garden....                                                                      

  9 "a seed in a fruit" or "a kernel of a fruit." It can refer to:                                                 

1) A seed:                                                                                                         

 • A small, hard, dry fruit formed by the ovary of...                                                              

 10 "a nut that has a single, hard seed enclosed within a shell." It is derived from the Arabic word "ʿnus",       
    meaning " kernel".                                                                                             

Verbosity:                                                                                                         

The word "nut"...